<a href="https://colab.research.google.com/github/ChitranshBose/CSE508_Winter2023_A2_47/blob/main/IR_A2_47_ASSIGN2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [375]:
# importing necessary libraries
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import  StandardScaler 
from sklearn import metrics as mt

import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

import re

from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [376]:
# reading data and naming the columns
data_train = pd.read_csv("BBC News Train.csv", on_bad_lines='skip', encoding="ISO-8859-1")
data_train.drop(['ArticleId'],inplace=True,axis=1)
data_train.head()


,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [377]:
data_train_temp = data_train.copy()
data_train_temp

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business
...,...,...
1485,double eviction from big brother model caprice...,entertainment
1486,dj double act revamp chart show dj duo jk and ...,entertainment
1487,weak dollar hits reuters revenues at media gro...,business
1488,apple ipod family expands market apple has exp...,tech


In [378]:
# lowering the case of the dataset
data_train['Category'] = data_train['Category'].str.lower()
data_train['Text'] = data_train['Text'].str.lower()
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [379]:
def removePunctuation(data):
  token = RegexpTokenizer(r"\w+")
  return token.tokenize(''.join(data))

In [380]:
data_train['Text'] = data_train['Text'].apply(removePunctuation)
data_train.head()

,Text,Category
0,"[worldcom, ex, boss, launches, defence, lawyer...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, in, 168m, payout, eighteen, fo...",business


In [381]:
stopwords = stopwords.words('english')
#stop_words = set(stopwords.words("english"))
def removeWords(tweet):
  
  output = []
  for word in tweet:
    if word not in stopwords:
      output.append(word)
  return output

In [382]:
data_train['Text'] = data_train['Text'].apply(removeWords)
data_train.head()

,Text,Category
0,"[worldcom, ex, boss, launches, defence, lawyer...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, 168m, payout, eighteen, former...",business


In [383]:
def lemmatize(data):
  output = []
  lst = ['a','r','n','v']
  wordNetLemmatizer = WordNetLemmatizer()
  posTag = pos_tag(data)
  for word, tag in posTag:
    pos = tag[0].lower()

    if pos not in lst:
      pos = 'n'
    output.append(wordNetLemmatizer.lemmatize(word,pos))
  return output

In [384]:
data_train['Text'] = data_train['Text'].apply(lemmatize)
data_train.head()

,Text,Category
0,"[worldcom, ex, bos, launch, defence, lawyer, d...",business
1,"[german, business, confidence, slide, german, ...",business
2,"[bbc, poll, indicate, economic, gloom, citizen...",business
3,"[lifestyle, governs, mobile, choice, faster, w...",tech
4,"[enron, boss, 168m, payout, eighteen, former, ...",business


In [385]:
def stemming(data):
  output = []
  stem = PorterStemmer()
  for word in data:
    output.append(stem.stem(word))
  return output

In [387]:
categories = data_train['Category'].unique()

term_freq = {}
class_freq = {}
num_class = len(categories)

In [388]:
def get_cat(categories, data):
  
  cat_business = data[data['Category'] == 'business']
    
  cat_tech = data[data['Category'] == 'tech']
    
  cat_politics = data[data['Category'] == 'politics']

  cat_sport = data[data['Category'] == 'sport']
    
  cat_entertain = data[data['Category'] == 'entertainment']
    
  
  return cat_business, cat_tech, cat_politics, cat_sport, cat_entertain
      

In [389]:
cat_business, cat_tech, cat_politics, cat_sport, cat_entertain = get_cat(categories,data_train)


In [390]:
def cal_tf(term_freq_data,data,cat_type):
  for text in data['Text']:
    for word in text:
      if word not in term_freq_data:
        term_freq_data[word]={}
      if cat_type not in term_freq_data[word]:
        term_freq_data[word][cat_type] = 0
      term_freq_data[word][cat_type]+=1
  return  term_freq_data  



In [391]:
def calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain):
  term_freq = cal_tf(term_freq, cat_business, "business")
  term_freq = cal_tf(term_freq, cat_tech, "tech")
  term_freq = cal_tf(term_freq, cat_politics, "politics")
  term_freq = cal_tf(term_freq, cat_sport, "sport")
  term_freq = cal_tf(term_freq, cat_entertain, "entertainment")
  return term_freq

In [392]:

term_freq = calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)
print(term_freq)

{'worldcom': {'business': 54}, 'ex': {'business': 10, 'tech': 2, 'politics': 35, 'sport': 4, 'entertainment': 10}, 'bos': {'business': 24, 'tech': 4, 'politics': 4, 'sport': 71, 'entertainment': 2}, 'launch': {'business': 39, 'tech': 116, 'politics': 39, 'sport': 8, 'entertainment': 25}, 'defence': {'business': 18, 'tech': 9, 'politics': 16, 'sport': 37, 'entertainment': 1}, 'lawyer': {'business': 26, 'tech': 8, 'politics': 19, 'sport': 7, 'entertainment': 15}, 'defend': {'business': 2, 'tech': 5, 'politics': 31, 'sport': 45, 'entertainment': 3}, 'former': {'business': 73, 'tech': 15, 'politics': 81, 'sport': 85, 'entertainment': 52}, 'chief': {'business': 156, 'tech': 46, 'politics': 58, 'sport': 23, 'entertainment': 15}, 'bernie': {'business': 8}, 'ebbers': {'business': 45}, 'battery': {'business': 1, 'tech': 23, 'sport': 3, 'entertainment': 1}, 'fraud': {'business': 66, 'tech': 15, 'politics': 23, 'entertainment': 1}, 'charge': {'business': 69, 'tech': 25, 'politics': 38, 'sport': 7

In [393]:
def cal_cf(term_freq,class_freq,data,cat_type):
  
  words = term_freq.keys()
  for word in words:
      if word not in class_freq:
        class_freq[word]=0
      if cat_type in term_freq[word]:
        class_freq[word]+=1
  return  class_freq  



In [394]:
def calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain):
  class_freq = cal_cf(term_freq, class_freq, cat_business, "business")
  class_freq = cal_cf(term_freq, class_freq, cat_tech, "tech")
  class_freq = cal_cf(term_freq, class_freq, cat_politics, "politics")
  class_freq = cal_cf(term_freq, class_freq, cat_sport, "sport")
  class_freq = cal_cf(term_freq, class_freq, cat_entertain, "entertainment")
  return class_freq


In [395]:
class_freq = calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)
print(class_freq)

{'worldcom': 1, 'ex': 5, 'bos': 5, 'launch': 5, 'defence': 5, 'lawyer': 5, 'defend': 5, 'former': 5, 'chief': 5, 'bernie': 1, 'ebbers': 1, 'battery': 4, 'fraud': 4, 'charge': 5, 'call': 5, 'company': 5, 'whistleblower': 1, 'first': 5, 'witness': 5, 'cynthia': 2, 'cooper': 4, 'head': 5, 'internal': 5, 'accounting': 1, 'alert': 5, 'director': 5, 'irregular': 2, 'practice': 5, 'u': 5, 'telecoms': 2, 'giant': 5, '2002': 5, 'warning': 5, 'lead': 5, 'collapse': 5, 'firm': 5, 'follow': 5, 'discovery': 4, '11bn': 2, 'â': 5, '5': 5, '7bn': 1, 'mr': 5, 'plead': 5, 'guilty': 5, 'conspiracy': 4, 'prosecution': 5, 'argue': 5, 'orchestrate': 2, 'series': 5, 'trick': 5, 'order': 5, 'employee': 5, 'hide': 5, 'expense': 5, 'inflate': 3, 'revenue': 5, 'meet': 5, 'wall': 4, 'street': 5, 'earnings': 3, 'estimate': 4, 'ms': 4, 'run': 5, 'consult': 5, 'business': 5, 'tell': 5, 'jury': 4, 'new': 5, 'york': 5, 'wednesday': 5, 'external': 3, 'auditor': 3, 'arthur': 5, 'andersen': 2, 'approve': 3, 'account': 5,

In [396]:
term_freq['eef']


{'business': 2}

In [397]:
def tf_icf(term_freq,class_freq):
  tf_icf_dict = {}
  for word in term_freq.keys():
      tf_icf_dict[word] = {}
      for category in categories:
          tf = term_freq[word].get(category, 0)
          # print(tf)
          # print(class_freq[word])
          # break
          cf = class_freq[word]
          icf = np.log10(num_class / cf)
        
          tf_icf_dict[word][category] = tf * icf
  return tf_icf_dict

In [398]:
#print(term_freq)
#print(class_freq)
tf_icf_dict = tf_icf(term_freq,class_freq)

In [399]:
# lst = []
# for x, y in zip(data_train['Text'],data_train['Category']):
#   lst_train = []
#   for word in x:
#     lst_train.append(tf_icf_dict[word][y])
#   lst.append(lst_train)

In [400]:
df = pd.DataFrame(tf_icf_dict)
df
# tf_icf_dict.keys()

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
business,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
tech,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
politics,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
sport,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
entertainment,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897


In [401]:
#names = tf_icf_dict.keys()
# ind = data_train.index
empty_dataframe = []
df1 = pd.DataFrame(empty_dataframe,columns = tf_icf_dict.keys(),index = data_train.index )
df1

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
def fill_data(df, df1, data_train):
  ind = data_train.index
  for i in range(len(ind)):
    cat = data_train.iloc[i]['Category']
    text = data_train.iloc[i]['Text']
 
    for word in text:
   
      df1.iloc[i][word] = df.loc[cat][word]
  
  return df1

In [403]:
df1 = fill_data(df, df1, data_train).copy()

In [404]:
df1

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897
1487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [405]:
df1 = df1.fillna(0)

In [406]:
df1.head(-5)

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1481,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1482,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1483,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [407]:
x = df1
y = data_train['Category']


In [408]:
x

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1486,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897
1487,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1488,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [409]:
# function to split the dataset into train and test
def training_testing_split(x,y,test_size):
  train_size = int(len(x)*(1-test_size))
  train_data_x = x[:train_size]
  train_data_y = y[:train_size]
  test_data_x = x[train_size:]
  test_data_y = y[train_size:]

  return train_data_x, test_data_x, train_data_y, test_data_y

In [410]:
# from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

In [411]:
x_train

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1039,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1040,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1041,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [412]:
from sklearn.naive_bayes import MultinomialNB

In [413]:
def naive_bayes(x_train,y_train,x_test):
  mb = MultinomialNB()
  mb.fit(x_train, y_train)
  
# making predictions on the testing set
  y_pred = mb.predict(x_test)
  return y_pred

In [414]:
lst_y_test = list(y_test)
# lst_x_test = list(x_test)
# lst_y_train = list(y_train)

In [415]:
def calc_acc(y_test, y_pred):
  acc = [1 if y_test[i]==y_pred[i] else 0 for i in range(len(y_test))]

  return np.sum(acc)/len(y_test)

In [416]:
y_pred = naive_bayes(x_train,y_train,x_test)
accuracy = calc_acc(lst_y_test,y_pred)
print("Accuracy: ",accuracy)

Accuracy:  0.9977628635346756


In [417]:
def calc_freq_cat(y_train):
  freq_cat = dict()
  prob = dict()
  num_docs  = len(y_train)

  for cat in y_train:
    if cat not in freq_cat:
      freq_cat[cat]=0
    freq_cat[cat]+=1

# print(freq_cat)

  for cat, freq in freq_cat.items():
    prob[cat] = freq/num_docs

  print(prob)



In [418]:
calc_freq_cat(y_train)

{'business': 0.22722914669223393, 'tech': 0.17833173537871524, 'politics': 0.18024928092042186, 'sport': 0.23873441994247363, 'entertainment': 0.17545541706615533}


In [421]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(lst_y_test,y_pred)
print(conf_mat)

[[99  0  0  0  0]
 [ 0 90  0  0  0]
 [ 1  0 85  0  0]
 [ 0  0  0 97  0]
 [ 0  0  0  0 75]]


In [422]:
def calc_tp_tf_fp_fn(conf_mat):
  tp = np.diag(conf_mat)
  fp = np.sum(conf_mat,axis = 0) - tp
  fn = np.sum(conf_mat,axis = 1) - tp
  tn = np.sum(conf_mat) - tp - fp - fn

  return tp, fp, fn, tn

In [423]:
def calc_prec(tp, fp, fn, tn):
  prec = np.sum(tp)/np.sum(tp+fp)
  return prec

In [424]:
def calc_rec(tp, fp, fn, tn):
  rec = np.sum(tp)/np.sum(tp+fn)
  return rec

In [425]:
tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)


Precision:  0.9977628635346756
Recall:  0.9977628635346756
F1 Score:  0.9977628635346756


In [426]:
data_train_temp

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business
...,...,...
1485,double eviction from big brother model caprice...,entertainment
1486,dj double act revamp chart show dj duo jk and ...,entertainment
1487,weak dollar hits reuters revenues at media gro...,business
1488,apple ipod family expands market apple has exp...,tech


In [427]:
data_train = data_train_temp.copy()



# lowering the case of the dataset
data_train['Category'] = data_train['Category'].str.lower()
data_train['Text'] = data_train['Text'].str.lower()
data_train.head()


,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [428]:

categories = data_train['Category'].unique()

term_freq = {}
class_freq = {}
num_class = len(categories)

  

In [429]:
data_train['Text'] = data_train['Text'].apply(removePunctuation)

data_train['Text'] = data_train['Text'].apply(removeWords)


data_train['Text'] = data_train['Text'].apply(stemming)


cat_business, cat_tech, cat_politics, cat_sport, cat_entertain = get_cat(categories,data_train)

term_freq = calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)

class_freq = calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)


In [430]:
data_train.head()

,Text,Category
0,"[worldcom, ex, boss, launch, defenc, lawyer, d...",business
1,"[german, busi, confid, slide, german, busi, co...",business
2,"[bbc, poll, indic, econom, gloom, citizen, maj...",business
3,"[lifestyl, govern, mobil, choic, faster, bette...",tech
4,"[enron, boss, 168m, payout, eighteen, former, ...",business


In [431]:
tf_icf_dict = tf_icf(term_freq,class_freq)

df = pd.DataFrame(tf_icf_dict)


empty_dataframe = []
df1 = pd.DataFrame(empty_dataframe,columns = tf_icf_dict.keys(),index = data_train.index )

df1 = fill_data(df, df1, data_train).copy()
df1 = df1.fillna(0)




In [432]:
x = df1
y = data_train['Category']


In [433]:
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)

accuracy = calc_acc(lst_y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(lst_y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.9977628635346756
Precision:  0.9977628635346756
Recall:  0.9977628635346756
F1 Score:  0.9977628635346756


In [434]:
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.4)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)

accuracy = calc_acc(lst_y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(lst_y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.9983221476510067
Precision:  0.9983221476510067
Recall:  0.9983221476510067
F1 Score:  0.9983221476510067


In [442]:
calc_freq_cat(y_train)

{'business': 0.22722914669223393, 'tech': 0.17833173537871524, 'politics': 0.18024928092042186, 'sport': 0.23873441994247363, 'entertainment': 0.17545541706615533}


In [435]:
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.5)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)

accuracy = calc_acc(lst_y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(lst_y_test,y_pred)
  #print(conf_mat)

tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

{'business': 0.24026845637583893, 'tech': 0.18791946308724833, 'politics': 0.16375838926174496, 'sport': 0.23624161073825503, 'entertainment': 0.17181208053691274}
Accuracy:  0.9986577181208054
Precision:  0.9986577181208054
Recall:  0.9986577181208054
F1 Score:  0.9986577181208054


In [436]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [437]:
x = data_train['Text']
y = data_train['Category']
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

# lst_y_test = list(y_test)

In [438]:
vect = TfidfVectorizer()
x_train_vect = [' '.join(data) for data in x_train]
x_test_vect = [' '.join(data) for data in x_test]


In [439]:
x_train_vect = vect.fit_transform(x_train_vect)
x_test_vect = vect.transform(x_test_vect)

In [440]:
mb = MultinomialNB()
mb.fit(x_train_vect, y_train)
y_test = list(y_test)
y_pred = mb.predict(x_test_vect)

In [441]:

accuracy = calc_acc(y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.970917225950783
Precision:  0.970917225950783
Recall:  0.970917225950783
F1 Score:  0.9709172259507831
